In [1]:
from __future__ import print_function, division
from Bio.pairwise2 import format_alignment
from operator import itemgetter
import os
import sys
import tempfile
import warnings
import pandas as pd0
import string
import pandas as pd
try:
    from Bio import pairwise2
    from Bio.SubsMat import MatrixInfo as matlist
except ImportError as exception:
    print("[!] Could not import Biopython modules", file=sys.stderr)
    raise exception

c:\Users\CPE\anaconda3\lib\site-packages\Bio\SubsMat\__init__.py:126: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  warnings.warn(


โค้ดหลัก

In [2]:
Hydrophobic_list =['F','W','I','L','V','A','M','C']
Neutral_list = ['Y','H','T','S','P','G']
Hydrophilic_list =['R','Q','K','N','E','D']
charged_list = ['K','R','H','E','D']
Uncharged_list = ['T','S','Q','N']
sim_list = ['DFASCHTNGGICLPNRCPGHMIQIGICFRPRVKCCRSW', 'MKFTIVFLLLACVFAMGVATPGKPRPYSPRPTSHPRPIRVRREALAIEDHLTQAAIRPPPILPA', 'MASTERNFLLLSLVVSALSGLVHRSDAAEISFGSCTPQQSDERGQCVHITSCPYLANLLMVEPKTPAQRILLSKSQCGLDNRVEGLVNRILVCCPQSMRGNIMDSEPTPSTRDALQQGDVLPGNDVCGFLFADRIFGGTNTTLWEFPWMVLLQYKKLFSETYTFNCGGALLNSRYVLTAGHCLASRELDKSGAVLHSVRLGEWDTRTDPDCTTQMNGQRICAPKHIDIEVEKGIIHEMYAPNSVDQRNDIALVRLKRIVSYTDYVRPICLPTDGLVQNNFVDYGMDVAGWGLTENMQPSAIKLKITVNVWNLTSCQEKYSSFKVKLDDSQMCAGGQLGVDTCGGDSGGPLMVPISTGGRDVFYIAGVTSYGTKPCGLKGWPGVYTRTGAFIDWIKQKLEP','EDLTVKIGDFGLATEKSRWSGSHQFEQLS','MVLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGHGKKVADALTNAVAHVDDMPNALSALSDLHAHKLRVDPVNFKLLSHCLLVTLAAHLPAEFTPAVHASLDKFLASVSTVLTSKYR', 'MTTCSRQFTSSSSMKGSCGIGGGIGAGSSRISSVLAGGSCRAPNTYGGGLSVSSSRFSSGGAYGLGGGYGGGFSSSSSSFGSGFGGGYGGGLGAGLGGGFGGGFAGGDGLLVGSEKVTMQNLNDRLASYLDKVRALEEANADLEVKIRDWYQRQRPAEIKDYSPYFKTIEDLRNKILTATVDNANVLLQIDNARLAADDFRTKYETELNLRMSVEADINGLRRVLDELTLARADLEMQIESLKEELAYLKKNHEEEMNALRGQVGGDVNVEMDAAPGVDLSRILNEMRDQYEKMAEKNRKDAEEWFFTKTEELNREVATNSELVQSGKSEISELRRTMQNLEIELQSQLSMKASLENSLEETKGRYCMQLAQIQEMIGSVEEQLAQLRCEMEQQNQEYKILLDVKTRLEQEIATYRRLLEGEDAHLSSSQFSSGSQSSRDVTSSSRQIRTKVMDVHDGKVVSTHEQVLRTKN']

In [3]:
def calculate_align_sequences(sequence_A, sequence_B, **kwargs):
    """
    Performs a global pairwise alignment between two sequences
    using the BLOSUM62 matrix and the Needleman-Wunsch algorithm
    as implemented in Biopython. Returns the alignment, the sequence
    identity and the residue mapping between both original sequences.
    """

    def _calculate_identity(sequenceA, sequenceB):
        """
        Returns the percentage of identical characters between two sequences.
        Assumes the sequences are aligned.
        """

        sa, sb, sl = sequenceA, sequenceB, len(sequenceA)
        c_matches = [sa[i] == sb[i] for i in range(sl)]
        matches = sum([1 for i in range(sl) if (sa[i] == sb[i])])
        # gapless_sl = sum([1 for i in range(sl) if (sa[i] != '-' and sb[i] != '-')])
        # gap_id = (100 * sum(matches)) / gapless_sl
        gaps_al = sum([1 for i in range(sl) if (sa[i] == '-' or sb[i] == '-')])
        return (matches, gaps_al)
        
    def _calculate_similarity(sequenceA, sequenceB):
        sa, sb, sal, sbl = sequenceA, sequenceB, len(sequenceA), len(sequenceB)
        c_sim = 0
        align = []
        for i in range(sal):
            if ((sa[i] in Hydrophobic_list and sb[i] in Hydrophobic_list ) or (sa[i] in Neutral_list and sb[i] in Neutral_list ) or (sa[i] in Hydrophilic_list and sb[i] in Hydrophilic_list) or (sa[i] in charged_list and sb[i] in charged_list)  or (sa[i] in Uncharged_list and sb[i] in Uncharged_list) or (sa[i] == sb[i])):
            # if ((sa[i] in Aliphatic_list and sb[i] in Aliphatic_list ) or (sa[i] in Hydrophobic_list and sb[i] in Hydrophobic_list ) or (sa[i] in Hydroxyl and sb[i] in Hydroxyl) or (sa[i] in charged_list and sb[i] in charged_list) or (sa[i] in Acidic and sb[i] in Acidic) or (sa[i] in Hydrophilic_list and sb[i] in Hydrophilic_list) or (sa[i] in a_list and sb[i] in a_list) or (sa[i] in b_list and sb[i] in b_list) or (sa[i] in c_list and sb[i] in c_list) or (sa[i] == sb[i]) or (sa[i] in a and sb[i] in a)):
                if (sa[i] != sb[i]):
                    align.append(':')
                elif (sa[i] == sb[i]):
                    align.append('|')
                elif ((sa[i] != sb[i]) and (sa[i] != '-') and (sb[i] != '-')):
                    align.append('.')
                c_sim = c_sim+1
            else:
                if ((sa[i] != sb[i]) and (sa[i] != '-') and (sb[i] != '-')):
                    align.append('.')
                else:  
                    align.append(' ')
        align_l = ''.join(align)
        return (align_l, c_sim)
    #
    matrix = kwargs.get('matrix', matlist.blosum62)
    gap_open = kwargs.get('gap_open', -10.0)
    gap_extend = kwargs.get('gap_extend', -0.5)

    alns = pairwise2.align.globalds(sequence_A, sequence_B,
                                    matrix, gap_open, gap_extend,
                                    penalize_end_gaps=(False, False) )
    seq_id_list = []
    x = 0
    # print(len(alns))
    for alignment in alns: 
        aligned_A, aligned_B, score, begin, end = alignment
        seq_id = _calculate_identity(aligned_A, aligned_B)
        seq_id_list.append(seq_id)
    inden = seq_id_list[0]
    for i in seq_id_list:
        if i <= inden:
            aligned_A, aligned_B, score, begin, end = alns[x]
        x=x+1
          
    # Calculate sequence identity
    matches, gaps_al = _calculate_identity(aligned_A, aligned_B)
    align_l, c_sim  = _calculate_similarity(aligned_A, aligned_B)

    c_dot = sum([1 for i in range(len(align_l)) if (align_l[i] == '.')])
    c_co = sum([1 for i in range(len(align_l)) if (align_l[i] == ':')])
    len_a, len_b = len(sequence_A), len(sequence_B)
    len_al = (len_a + len_b) - (matches + c_dot + c_co)

    gaps = (100 * gaps_al)/ len_al
    sim = (100 * c_sim) / len_al
    iden = (100 * matches) / len_al

    # return (aligned_A,align_list, aligned_B), 'score: %.1f' %score, 'identity: %.1f' %seq_id,'similarity: %.1f' %sim_p
    return (aligned_A,align_l, aligned_B), float(f'{sim:,.2f}'), float(f'{iden:,.2f}'), float(f'{gaps:,.2f}')

หน้าเว็ปแสดงค่า sim ในส่วนของ Feature แต่ละตัว

In [4]:
# ใช้หน้าpredic
def align_sequences(sequence_B):
    # sim_list = ['DFASCHTNGGICLPNRCPGHMIQIGICFRPRVKCCRSW', 'MKFTIVFLLLACVFAMGVATPGKPRPYSPRPTSHPRPIRVRREALAIEDHLTQAAIRPPPILPA', 'MASTERNFLLLSLVVSALSGLVHRSDAAEISFGSCTPQQSDERGQCVHITSCPYLANLLMVEPKTPAQRILLSKSQCGLDNRVEGLVNRILVCCPQSMRGNIMDSEPTPSTRDALQQGDVLPGNDVCGFLFADRIFGGTNTTLWEFPWMVLLQYKKLFSETYTFNCGGALLNSRYVLTAGHCLASRELDKSGAVLHSVRLGEWDTRTDPDCTTQMNGQRICAPKHIDIEVEKGIIHEMYAPNSVDQRNDIALVRLKRIVSYTDYVRPICLPTDGLVQNNFVDYGMDVAGWGLTENMQPSAIKLKITVNVWNLTSCQEKYSSFKVKLDDSQMCAGGQLGVDTCGGDSGGPLMVPISTGGRDVFYIAGVTSYGTKPCGLKGWPGVYTRTGAFIDWIKQKLEP','EDLTVKIGDFGLATEKSRWSGSHQFEQLS','MVLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGHGKKVADALTNAVAHVDDMPNALSALSDLHAHKLRVDPVNFKLLSHCLLVTLAAHLPAEFTPAVHASLDKFLASVSTVLTSKYR', 'MTTCSRQFTSSSSMKGSCGIGGGIGAGSSRISSVLAGGSCRAPNTYGGGLSVSSSRFSSGGAYGLGGGYGGGFSSSSSSFGSGFGGGYGGGLGAGLGGGFGGGFAGGDGLLVGSEKVTMQNLNDRLASYLDKVRALEEANADLEVKIRDWYQRQRPAEIKDYSPYFKTIEDLRNKILTATVDNANVLLQIDNARLAADDFRTKYETELNLRMSVEADINGLRRVLDELTLARADLEMQIESLKEELAYLKKNHEEEMNALRGQVGGDVNVEMDAAPGVDLSRILNEMRDQYEKMAEKNRKDAEEWFFTKTEELNREVATNSELVQSGKSEISELRRTMQNLEIELQSQLSMKASLENSLEETKGRYCMQLAQIQEMIGSVEEQLAQLRCEMEQQNQEYKILLDVKTRLEQEIATYRRLLEGEDAHLSSSQFSSGSQSSRDVTSSSRQIRTKVMDVHDGKVVSTHEQVLRTKN']
    c = 0
    list_sim_align = []
    for i in sim_list:
        align = calculate_align_sequences(i, sequence_B)
        list_sim_align.append(align[1])
        #ค่าในlist --> defensin,Drosocin,Spaetzle,B-RAF,hemoglobin,keratin
    return list_sim_align

In [5]:
a = 'DFASCHTNGGICLPNRCPGHMIQIGICFRPRVKCCRSW'
sequence_A = 'DFASCHTNGGICLPNRCPGHMIQIGICFRPRVKCCRSW'
sequence_B = 'MMNVYYFHHQADELLGTASEQFLGKSVKEIKQTILQTLEGHLRAILGTLTVEEVYKDRDQFAALVREVAAPDVGRMGIEILSFTIKDVYDDVQYLASLGKAQTAVVKRDADAGVAEANRDAGIREAECEKSAMDVKYSTDTKIEDNTRMYKLQKANFDQEINTAKAESQLAYELQAAKIRQRIRNEEIQIEVVERRKQIEIESQEVQRKDRELTGTVKLPAEAEAFRLQTLAQAKQCQTIEGARAEAERIRKIGSAEAHAIELVGKAEAERMRMKAHVYKQYGDAAIMNIVLESLPKIAAEVAAPLAKTDEIVLIGGNDNITNDVTRLVAQLPPSINALTGVDLSKVLSKIPGAKA'

In [6]:
align_sequences(sequence_A)

[100.0, 9.3, 4.13, 8.2, 6.67, 0.8]

In [7]:
sequence_A = 'DFASCHTNGGICLPNRCPGHMIQIGICFRPRVKCCRSW'
sequence_B = 'MMNVYYFHHQADELLGTASEQFLGKSVKEIKQTILQTLEGHLRAILGTLTVEEVYKDRDQFAALVREVAAPDVGRMGIEILSFTIKDVYDDVQYLASLGKAQTAVVKRDADAGVAEANRDAGIREAECEKSAMDVKYSTDTKIEDNTRMYKLQKANFDQEINTAKAESQLAYELQAAKIRQRIRNEEIQIEVVERRKQIEIESQEVQRKDRELTGTVKLPAEAEAFRLQTLAQAKQCQTIEGARAEAERIRKIGSAEAHAIELVGKAEAERMRMKAHVYKQYGDAAIMNIVLESLPKIAAEVAAPLAKTDEIVLIGGNDNITNDVTRLVAQLPPSINALTGVDLSKVLSKIPGAKA'

In [8]:
list_val_sim = align_sequences(sequence_B)
#type ของค่าใน list คือ float
#ค่าในlistเรียงแบบนี้ --> defensin,Drosocin,Spaetzle,B-RAF,hemoglobin,keratin
#อยากได้ค่าตัวไหน เรียกค่าของตำแหน่งใน list_val_sim เช่น
#ค่าของ Drosocin อยู่ใน list ตำแหน่งที่ 2 index ที่ 1 
# ใช้คำสั่ง --> list_val_sim[1]

In [9]:
#Ex.
list_val_sim

[5.28, 7.41, 10.0, 4.76, 18.09, 25.64]

หน้าเว็ป align sequences ของ user

In [10]:
align_sequences_user = calculate_align_sequences(sequence_A, sequence_B)
#ใช้codeเดิม แค่เรียกคนละ def
#หน้าเว็ป align sequences ของ user
# ค่าจะเรียงเป็น --> วิธี align, sim, iden, gap
#อยากได้ค่าตัวไหน เรียกค่าของตำแหน่งใน align_sequences_user เช่น
#ค่าของ sim อยู่ใน align_sequences_user ตำแหน่งที่ 2 index ที่ 1 
#type ของค่าใน sim, iden, gap คือ float
# ใช้คำสั่ง --> align_sequences_user[1]

In [11]:
#Ex.
align_sequences_user[0][1]

'                                                                                                                                                                                                              :....:.:|::.||.::....:|.    ..::|.|:::                                                                                                                    '